Assista estas aulas para a teoria base como complemento:
- https://www.youtube.com/watch?v=NSrOa4qnhvk
- https://www.youtube.com/watch?v=2vSePCL156w
- https://www.youtube.com/watch?v=gRb4OnpeoEg


In [83]:
# Carregando as bibliotecas

import pandas as pd
import numpy as np
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from math import sqrt

from statsmodels.tsa.stattools import adfuller

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [86]:
start = "2018-06-30"
end = "2022-10-31"
#end = datetime.todtickersay().strftime("%Y-%m-%d")

tickers = ["BOVA11.SA", "PETR4.SA", "VALE3.SA", "ITUB4.SA"
           , "CSNA3.SA", "BBAS3.SA", "WEGE3.SA", "B3SA3.SA" ,  "JBSS3.SA"
           , "ITSA4.SA", "CMIG4.SA", "SBSP3.SA", "CCRO3.SA", "UGPA3.SA"
           , "MULT3.SA", "BPAC11.SA", "EQTL3.SA", "LREN3.SA", "ENBR3.SA"
           , "TAEE11.SA", "SUZB3.SA", "BBDC4.SA", "BBSE3.SA", "ECOR3.SA"
           , "RADL3.SA"]

ativos = []
for ticker in tickers:
    try:
        print(ticker)
        cotacoes = yf.download(ticker, start, end)["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        ativos.append(cotacoes)
    except:
        pass
    base_ativos = pd.concat(ativos, axis = 1)
base_ativos.sort_index(inplace = True)
base_ativos.head()
base_ativos.dropna(inplace = True)

BOVA11.SA
[*********************100%***********************]  1 of 1 completed
PETR4.SA
[*********************100%***********************]  1 of 1 completed
VALE3.SA
[*********************100%***********************]  1 of 1 completed
ITUB4.SA
[*********************100%***********************]  1 of 1 completed
CSNA3.SA
[*********************100%***********************]  1 of 1 completed
BBAS3.SA
[*********************100%***********************]  1 of 1 completed
WEGE3.SA
[*********************100%***********************]  1 of 1 completed
B3SA3.SA
[*********************100%***********************]  1 of 1 completed
JBSS3.SA
[*********************100%***********************]  1 of 1 completed
ITSA4.SA
[*********************100%***********************]  1 of 1 completed
CMIG4.SA
[*********************100%***********************]  1 of 1 completed
SBSP3.SA
[*********************100%***********************]  1 of 1 completed
CCRO3.SA
[*********************100%***********************]  1 

In [87]:
base_ativos.keys()

Index(['BOVA11.SA', 'PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'CSNA3.SA', 'BBAS3.SA',
       'WEGE3.SA', 'B3SA3.SA', 'JBSS3.SA', 'ITSA4.SA', 'CMIG4.SA', 'SBSP3.SA',
       'CCRO3.SA', 'UGPA3.SA', 'MULT3.SA', 'BPAC11.SA', 'EQTL3.SA', 'LREN3.SA',
       'ENBR3.SA', 'TAEE11.SA', 'SUZB3.SA', 'BBDC4.SA', 'BBSE3.SA', 'ECOR3.SA',
       'RADL3.SA'],
      dtype='object')

In [88]:
# Função para encontrar pares cointegrados

def encontra_pares(base):
    n = base.shape[1] # Recebe total de ativos
    pvalor_matriz = np.ones((n, n))
    keys = base.keys() #Keys() retorna a chave/índice da base de ativos
    pares = []
    for i in range(n):
        for j in range(i+1, n):
            a1 = base[keys[i]]
            a2 = base[keys[j]]
            resultado = coint(a1, a2)
            score = resultado[0]
            pvalor = resultado[1]
            pvalor_matriz[i, j] = pvalor
            if pvalor < 0.05:
                pares.append((keys[i], keys[j]))
    return pvalor_matriz, pares

In [89]:
# Encontremos então, os pares

pvalores, pares = encontra_pares(base_ativos)

In [90]:
pares

[('BOVA11.SA', 'SUZB3.SA'),
 ('ITSA4.SA', 'MULT3.SA'),
 ('SBSP3.SA', 'BBSE3.SA'),
 ('BPAC11.SA', 'EQTL3.SA')]

In [106]:
pares[3]

('BPAC11.SA', 'EQTL3.SA')

In [107]:
a1 = pares[2][0]
a2 = pares[2][1]

In [108]:
x_train = sm.add_constant(base_ativos[a1])
y_train = base_ativos[a2]
regressao = sm.OLS(y_train, x_train).fit()

print(regressao.params)
print("-----------------------------")
print(regressao.params[a1])

beta = regressao.params[a1]

const       10.033289
SBSP3.SA     0.293095
dtype: float64
-----------------------------
0.29309524002251913


In [109]:
# Esse beta preferencialmente tem que ser positivo...

In [110]:
# Normaliza o resíduo pelo z-score

residuo = y_train - (regressao.params[0] + beta*x_train[a1])


def normaliza_zscore(x):
    return (x - x.mean())/np.std(x)


residuo_norm = normaliza_zscore(residuo)

In [111]:
# Atenção: esta parte não está presente na aula gravada e precisa ser considerada

res_mean = residuo.mean()
res_std = np.std(residuo)

In [112]:
residuo_norm

Date
2018-07-02   -0.022716
2018-07-03    0.045052
2018-07-04   -0.013534
2018-07-05   -0.043755
2018-07-06   -0.100603
                ...   
2022-10-24    0.285735
2022-10-25    0.134897
2022-10-26    0.199456
2022-10-27    0.591344
2022-10-28    0.444233
Length: 1060, dtype: float64

In [113]:
#Visualiza o z-score

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = residuo_norm.index
                         , y = residuo_norm
                         , name = "Z Score"
                         ,line = dict(color = "black"))
              , row = 1, col = 1)

fig.add_hline(y = 2, line_width = 1, line_dash = "dash", line_color = "green")
#fig.add_hline(y = 1, line_width = 1, line_dash = "dash", line_color = "green")

fig.add_hline(y = -2, line_width = 1, line_dash = "dash", line_color = "red")
#fig.add_hline(y = -1, line_width = 1, line_dash = "dash", line_color = "red")


fig.update_layout(height = 600, width = 800
                  , title_text = "Z-Score Long & Short por cointegração " + a1 + " - "+ a2
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Z-Score"
                  , showlegend = False
                  , legend_title = "Carteira"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "x")

fig.show()

In [114]:
# Mas estes resíduos são estacionários?

adf_diff1 = adfuller(residuo_norm)
print("ADF Test Statistic: %.2f" % adf_diff1[0])
print("5%% Critical Value: %.2f" % adf_diff1[4]['5%'])
print("p-value: %.2f" % adf_diff1[1])

ADF Test Statistic: -3.79
5% Critical Value: -2.86
p-value: 0.00


In [115]:
# Vamos pegar a base recente

start = "2021-11-01"
end = datetime.today().strftime("%Y-%m-%d")

tickers = [a1, a2]

ativos_ls = []
for ticker in tickers:
    try:
        print(ticker)
        cotacoes = yf.download(ticker, start, end)["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        ativos_ls.append(cotacoes)
    except:
        pass
    base_ls = pd.concat(ativos_ls, axis = 1)
base_ls.sort_index(inplace = True)
base_ls.head()
base_ls.dropna(inplace = True)

SBSP3.SA
[*********************100%***********************]  1 of 1 completed
BBSE3.SA
[*********************100%***********************]  1 of 1 completed


In [116]:
# Recalcular os resíduos
# Atenção, pequena modificação em relação ao código da aula gravada

residuo_ls = base_ls[a2] - (regressao.params[0] + beta*base_ls[a1])


def normaliza_zscore_ls(x):
    return (x - res_mean)/res_std


residuo_norm_ls = normaliza_zscore_ls(residuo_ls)

In [117]:
# Agora vamos avaliar os resultados
# Atenção, pequena modificação em relação ao código da aula gravada

resultados = pd.concat([residuo_norm_ls, residuo_ls
                        , base_ls[a1].pct_change()
                       ,base_ls[a2].pct_change()], axis = 1)
resultados.columns = ["ZScore", "Residuo", a1, a2]

# Vamos fazer o trade com 1 desvio padrão
dev = 1.2


# Resíduo = a2 - (beta0 + beta*a1)
# Se o resíduo é negativo, você vai long no spread
# Significa que a1 é maior do que a2, então você vende a1 e compra a2
# Em qual proporção? No beta. 1 ação de a2 para beta (X) ações de a1
# Detalhes aqui: https://www.youtube.com/watch?v=2vSePCL156w&t=238s

# Se o resíduo for positivo, você vai short no spread
# Sifnigica que a2 é maior do que a1, então você compra ai e vende a2

resultados["sinal"] = 0

resultados["sinal"] = np.where(resultados["ZScore"] < -dev, (resultados[a2] - beta*resultados[a1])
                              , np.where(resultados["ZScore"] > dev, (-resultados[a2] + beta*resultados[a1])
                                        , 0))

resultados["retornos_acc"] = resultados["sinal"].cumsum()

In [118]:
resultados

,ZScore,Residuo,SBSP3.SA,BBSE3.SA,sinal,retornos_acc
Date,,,,,,
2021-11-01,-0.316955,-0.579623,NaN,NaN,0.000000,0.000000
2021-11-03,-0.179951,-0.329080,0.024349,0.025397,0.000000,0.000000
2021-11-04,-0.043362,-0.079298,-0.021858,0.001327,0.000000,0.000000
2021-11-05,-0.003484,-0.006371,0.010335,0.008834,0.000000,0.000000
2021-11-08,0.069320,0.126767,-0.014929,-0.000876,0.000000,0.000000
...,...,...,...,...,...,...
2023-05-12,4.862204,8.891621,0.072637,0.005593,0.015696,-0.379163
2023-05-15,3.695101,6.757313,0.007503,-0.059134,0.061333,-0.317831
2023-05-16,3.932967,7.192304,-0.027688,0.000311,-0.008426,-0.326257


In [105]:
#Visualiza o resultado do L&S por cointegração

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = resultados.index
                         , y = resultados["retornos_acc"]*100
                         , name = "Z Score"
                         ,line = dict(color = "black"))
              , row = 1, col = 1)

fig.update_layout(height = 600, width = 800
                  , title_text = "Resultado - Long & Short por cointegração " + a1 + " - "+ a2
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Retorno (%)"
                  , showlegend = False
                  , legend_title = "Carteira"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "x unified")

fig.show()